In [1]:
#code to enable voila, the web page of our app
# !jupyter nbextension enable --py widgetsnbextension
# !jupyter serverextension enable voila

In [2]:
#imports and third party libraries
import mysql.connector as connection
import pandas as pd
from numpy import random, where
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as ex
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.inspection import permutation_importance

import warnings
warnings.filterwarnings('ignore')
import os
import time
from datetime import datetime
import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display, clear_output, HTML, Image
import IPython

from datetime import date
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
# front-end portion
def webPage(newdf, days):
    
    #anomaly button widget
    anomaly_btn = widgets.Button(
                description='Incoming Reading',
                tooltip='Send',
                layout=Layout(width='150px', height='30px'),
                style=dict(button_color='#ADD8E6',
                           font_weight='bold',
                           text_color='black'))
    
    #importing and embedding gifs
    fgif = widgets.HTML(value='<img src="fan.gif" width="100" align="center">')
    fgif.layout.display = 'none'
    sgif = widgets.HTML(value='<img src="sending.gif" width="50" align="center">')
    sgif.layout.display = 'none'
    
    #define output to display on button click
    i_out = widgets.Output()

    #on click event
    def on_ibutton_clicked(event):
        with i_out:
            i_out.clear_output()
            h_out.clear_output()
            #read the last line in mysql database (latest entry)
            testdf = pd.read_sql_query("SELECT * FROM SensorData WHERE id=(SELECT max(id) FROM SensorData)", mydb)
            #run the line through naive bayes
            result = testData(newdf, testdf)
            
            #if the outcome of the analysis is 1, it detected it as an anomaly
            if result == 1:
                print("Anomaly Detected!")
                print("Recalibrating Temperature Control System...")
                fgif.layout.display = 'block'
                time.sleep(4)
                fgif.layout.display = 'none'
                print("Calibration complete.")
                print("\nSending alert to Employees...")
                sgif.layout.display = 'block'
                time.sleep(3)
                print("Alert sent.")
                sgif.layout.display = 'none'
            # if 0, it's normal
            else:
                print("All Good!")
    
    #history button widget
    history_btn = widgets.Button(
                    description='View History',
                    tooltip='Send',
                    layout=Layout(width='150px', height='30px'),
                    style=dict(button_color='#D3D3D3',
                               font_weight='bold',
                               text_color='black'))
    #define output to display on button click
    h_out = widgets.Output()

    #on click event
    def on_hbutton_clicked(event):
        with h_out:
            h_out.clear_output()
            i_out.clear_output()
            #get list from a separate table that stores all previous analysed readings
            latestdf = pd.read_sql_query("SELECT * FROM Classified", mydb)
            if latestdf.empty:
                print('History has been cleared.')
            else:
                #for every reading in the db, print our anomaly class attribute and the timestamp
                for i, j in zip(latestdf.Class, latestdf.Timestamp):
                    print(i, j)

    #checking for onclick events
    anomaly_btn.on_click(on_ibutton_clicked)
    history_btn.on_click(on_hbutton_clicked)
    
    # put buttons side by side in horizontal box
    hbox_anomlyBtns = widgets.HBox([anomaly_btn, history_btn])
    #vertical layout of buttons and outputs
    vbox_anomaly = widgets.VBox([hbox_anomlyBtns, i_out, fgif, sgif, h_out])

    #view trend button widget
    trend_btn = widgets.Button(
                    description='View Graph',
                    tooltip='Send',
                    layout=Layout(width='150px', height='30px'),
                    style=dict(button_color='#D3D3D3',
                               font_weight='bold',
                               text_color='black'
                              )
                )

    t_out = widgets.Output()

    def on_button_clicked(event):
        #show graph picture
        image_headline.layout.visibility = 'visible'

        with t_out:
            t_out.clear_output()
            #call function to plot graph based on dataset
            genGraph(newdf, days)
    
    #open and display graph that was saved as png image
    file = open("trend.png", "rb")
    image = file.read()
    image_headline = widgets.Image(
                        value=image,
                        format='jpg',
                        width='400',
                        margin='100px 0 0 0'
                    )
    image_headline.layout.visibility = 'hidden'
    
    trend_btn.on_click(on_button_clicked)
    
    #more layouts...
    vbox_trend = widgets.VBox([trend_btn, image_headline, t_out])

    text_0 = widgets.HTML(value="<h1>Data Centre Monitoring System</h1>")
    text_1 = widgets.HTML(value="<br><h2>Anomaly Detection</h2>")
    text_2 = widgets.HTML(value="<br><h2>Past Month's Trend</h2>")

    vbox_text = widgets.VBox([text_0, text_1, vbox_anomaly, text_2, vbox_trend])

    page = widgets.HBox([vbox_text])
    
    return page

In [4]:
#plot past month trend graph based on dataset
def genGraph(df, period):
    plt.ioff()
    fig = plt.figure()
    ax = fig.add_axes([1,1,1,1])
    ax.plot(df.tail(period))
    ax.legend(['CPU Usage', 'RAM Usage', 'Download Speed', 'Upload Speed', 'Temperature', 'Humidity'])
    fig.savefig('trend.png', bbox_inches='tight')    
    plt.close()

In [5]:
#function to take raw dataset, remove outliers
def analyse(df):
    global x, y
    #features as x variable
    x=df.iloc[:,1:7]
    x=StandardScaler().fit_transform(x) # normalize the variables
    y=df['Class'] # assign y variable - the target 

#     performing PCA on dataset to reduce dimensions
    pca=PCA(n_components=2) #two PCA components chosen
    PC=pca.fit_transform(x)
    principalDF=pd.DataFrame(data=PC,columns=['pc1','pc2'])
    finalDf = pd.concat([principalDF, df[['Class']]], axis = 1)

    PCloadings = pca.components_.T * np.sqrt(pca.explained_variance_)
    components=df.columns.tolist()
    components=components[1:7]
    loadingdf=pd.DataFrame(PCloadings,columns=('PC1','PC2'))
    loadingdf["variable"]=components
    
    Xfinal=finalDf[['pc1','pc2']]
    yfinal=finalDf['Class']

    #DBScan chosen as yielded most accurate results
    dbscan=DBSCAN()
    model = dbscan.fit(Xfinal)

    colors = model.labels_

    neigh = NearestNeighbors(n_neighbors=2)
    nbrs = neigh.fit(Xfinal)
    distances, indices = nbrs.kneighbors(Xfinal)

    data = Xfinal
    model = DBSCAN(eps = 0.4, min_samples = 15).fit(data)
    
#     #uncomment below two lines to see clusters
#     colors = model.labels_
#     plt.scatter(data['pc1'], data['pc2'], c = colors)

    #list of outliers indexes
    outliers_idx = data[model.labels_ == -1].index
    
    return outliers_idx

In [6]:
def testData(newdf, testdf): #function to train new dataset and classify new entries as anomaly or not
    data = newdf.iloc[:,1:7]
    #features from training dataset as x train
    X_train = data
    #Class attribute from training dataset as y train
    y_train = newdf.Class
    #features from latest reading as x test
    X_test = testdf.iloc[:,1:7]
    #Class Attribute from latest reading as y test
    y_test = testdf.Class

    #using gaussian naive bayes to classify new entry
    gnb = GaussianNB()
    gnb.fit(X_train, y_train.values.ravel())
    y_pred = gnb.predict(X_test)
    
    #giving outcomes a label
    if y_pred[0] == 0:
        attr = "Normal"
    else:
        attr = "Anomaly"
    #making list for easy inout into sql
    l = []
    for i in X_test.values[0]:
        l.append(i)
    l.append(attr)
    
    #connect to sql and write outcome of new reading in a table
    mycursor = mydb.cursor()
    sql = "INSERT INTO Classified (CPUUsage, RAMUsage, DownloadSpeed, UploadSpeed, Temperature, Humidity, Class, Timestamp) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
    val = (float(l[0]), float(l[1]), float(l[2]), float(l[3]), float(l[4]), float(l[5]), str(l[6]), datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))
    mycursor.execute(sql, val)
    mydb.commit()
    
    ##uncomment the code below to see acccuracy of outcome
#     print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
#     imps = permutation_importance(gnb, X_train, y_prob)
#     print(imps.importances)
    return y_pred[0]

In [8]:
if __name__ == "__main__":

    # establish connection to sql database
    mydb = connection.connect(host="54.179.115.76", database = 'sensor',user="staff", passwd="password",use_pure=True)
    #reading file of real raw data
    rawdf = pd.read_csv('dataset.csv')
    #DBScan function
    outliers_idx = analyse(rawdf)
    #remove outliers from raw dataset for more accurate anomaly detection
    newdf = rawdf.drop(outliers_idx)
    
    #getting today's month for plotting past month's trend
    todays_month = date.today().month
    days = ''
    #find num of days in this month
    if todays_month == 2:
        days = 28
    elif todays_month == 4 or 6 or 9 or 11:
        days = 30
    else:
        days = 31
    
    #plot trend grpah
    genGraph(newdf, days)
    #display webpage
    page = webPage(newdf, days)
    display(page)